In [1]:
import numpy as np
from qiskit import *
from math import pi
from qiskit.visualization import *
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute, BasicAer
from qiskit.compiler import transpile
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.quantum_info import process_fidelity
from qiskit.extensions import RXGate, XGate, CXGate
from qiskit.ignis.mitigation.measurement import (complete_meas_cal,CompleteMeasFitter)

/tmp/ipykernel_22022/3599017300.py:11: DeprecationWarning: The qiskit.ignis package is deprecated and has been supersceded by the qiskit-experiments project. Refer to the migration guide: https://github.com/Qiskit/qiskit-ignis#migration-guide on how to migrate to the new project.
  from qiskit.ignis.mitigation.measurement import (complete_meas_cal,CompleteMeasFitter)


In [2]:
IBMQ.save_account(token="40903e935c997971dbac23a512b0c89305b35e083cc67d595f2eb38651a3af6987418780564641e75141632fc5df363647d83c784c0024fd3305f5f9b50a1a73")

configrc.store_credentials:WARNING:2022-06-01 20:34:12,059: Credentials already present. Set overwrite=True to overwrite.


In [3]:
IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [4]:
provider = IBMQ.get_provider(hub='ibm-q-research', group='independent-12', project='main')

In [5]:
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-research', group='independent-12', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q-research', group='independent-12', project='main')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q-research', group='independent-12', project='main')>,
 <IBMQBackend('ibmq_bogota') from IBMQ(hub='ibm-q-research', group='independent-12', project='main')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q-research', group='independent-12', project='main')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q-research', group='independent-12', project='main')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q-research', group='independent-12', project='main')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q-research', group='independent-12', project='main')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q-research', group='independent-12', project='main')>,
 <IBMQSimulator('simulator_extended_stabilizer') 

In [6]:
def A_gate(theta, qubits, circ, phi=0, l = 0, f = True):
    a = qubits[0] #this will be the first qubit
    b = qubits[1] #this will be the second
    
    if f == True:
        circ.cx(b,a) #I add the first CNOT, for which qubit b is the control and qubit a is the target
    else: 
        circ.x(a)
    #now comes the R dagger gate which has two parameters: theta and phi
    #but in our case phi is always zero
    ry1 = r"Y_"+str(l)+"^-1"
    ry  = r"Y_"+str(l)
    rz = r"Z_"+str(l)
    rz1 = r"Z_"+str(l)+"^-1"
    circ.ry(-(theta+np.pi/2),b, label = ry1) #rotation around the y axis
    circ.rz(-(phi+np.pi),b) #rotation around the z axis 
    
    #now the second CNOT for which target: b, control:a
    circ.cx(a,b)
    
    #and now the R gate for which we use the same two parameters
    circ.rz((phi+np.pi),b)
    circ.ry((theta+np.pi/2),b, label = ry)
    
    #and the last CNOT
    circ.cx(b,a)
    
    #we dont need to return the circuit itself because it is now updated

In [7]:
def circ_maker(n, qubits, measure = False, f = True): 
    
    theta1 = 3*np.pi/4
    theta2 = -2.97167
    theta3 = -2.18628 
    theta4 = np.pi
    theta5 = 5.23599
    
    a = qubits[0]
    b = qubits[1]
    c = qubits[2]
    d = qubits[3]

    qreg = QuantumRegister(n)
    creg = ClassicalRegister(4)
    circ = QuantumCircuit(qreg, creg)
    circ.x(b)
    circ.x(d)
    
    A_gate(theta=theta1, qubits = [a,b], circ = circ, l = 1, f = f)
    A_gate(theta=theta2, qubits = [c,d], circ = circ, l = 2, f = f)
    A_gate(theta=theta3, qubits = [b,c], circ = circ, l = 3)
    A_gate(theta=theta4, qubits = [a,b], circ = circ, l = 4)
    A_gate(theta=theta5, qubits = [c,d], circ = circ, l = 5)
    A_gate(theta=theta3, qubits = [b,c], circ = circ, l = 3)

    if measure == True:
        circ.measure([a,b,c,d], [0,1,2,3])
    else: 
        pass 
    
    return circ,qreg, creg

In [8]:
def class_fid(dict1, dict2 = None, comp_to_Cab = False): 
    
    dict_perfect = {
    "0011" : 1/3,
    "1100" : 1/3,
    "0101" : 1/12,
    "0110" : 1/12,
    "1001" : 1/12,
    "1010" : 1/12
    }
    
    if comp_to_Cab == True: 
        dict2 = dict_perfect
    else: 
        pass
    fid = 0 
    len1 = 0
    len2 = 0
    for key in dict1: 
        len1 += dict1[key]
    for key in dict2: 
        len2 += dict2[key]
        
    for key in dict1: 
        if key in dict2: 
            fid += (dict1[key]* dict2[key]/(len1*len2))**0.5
        else: 
            pass
    return fid

In [9]:
def mitigation(backend, n, qubits, shots, direct_counts):
   
    qr = QuantumRegister(n)
    meas_calibs, state_labels = complete_meas_cal(qubit_list = qubits, qr=qr, circlabel='mcal')
    cal_job = execute(meas_calibs, backend=backend, shots=shots)
    cal_results = cal_job.result()
    meas_fitter = CompleteMeasFitter(cal_results, state_labels, circlabel='mcal')
    meas_filter = meas_fitter.filter
    mitigated_counts = meas_filter.apply(direct_counts) 
    
    return mitigated_counts, meas_filter

In [10]:
def saver(filename, backend_name, time, shots, qubits, direct_counts, mitigated_counts, meas_filter):  
    with open (filename+".txt", "w") as f: 
        f.write("Backend: "+backend_name+"\n")
        f.write("date: "+ str(time)+"\n")
        f.write("Qubits in use: "+str(qubits)+"\n")
        f.write("Shots: "+str(shots)+"\n")
        f.write("\n")
        f.write("Classical fidelities:"+"\n")
        f.write("\t"+"direct: "+str(class_fid(dict1=direct_counts, comp_to_Cab = True))+"\n")
        f.write("\t"+"mitigated:"+str(class_fid(dict1=mitigated_counts, comp_to_Cab=True))+"\n")
        f.write("Direct counts: "+"\n")
        for key in direct_counts:
            f.write(str(key)+": "+str(direct_counts[key])+"\n")

        f.write("Mitigated_counts: "+"\n")
        for key in mitigated_counts:
            f.write(str(key)+": "+str(mitigated_counts[key])+"\n")
                

        f.write("Readout error mitigation matrix:"+"\n")
        for line in meas_filter.cal_matrix:
            f.write(str(line)+"\n")

## Test 1 

In [11]:
backend = Aer.get_backend('qasm_simulator')
backend_name = 'Aer qasm_simulator'
time = "2021.04.23"
n=5
shots = 8192
qubits = np.array([0,1,2,3])

In [12]:
circ,qreg,creg = circ_maker(n = n, qubits=qubits, f = False)
#tr_circ = transpile(circ ,backend = backend, optimization_level = 3)

circ.draw()

┌───┐                                                          ┌───┐     »
q0_0: ┤ X ├──────────────────────────────■───────────────────────────┤ X ├─────»
      ├───┤┌───────────────┐ ┌────────┐┌─┴─┐┌───────┐ ┌───────────┐  └─┬─┘┌───┐»
q0_1: ┤ X ├┤ Y_1^-1(-5π/4) ├─┤ Rz(-π) ├┤ X ├┤ Rz(π) ├─┤ Y_1(5π/4) ├────■──┤ X ├»
      ├───┤└───────────────┘ └────────┘└───┘└───────┘ └───────────┘  ┌───┐└─┬─┘»
q0_2: ┤ X ├──────────────────────────────■───────────────────────────┤ X ├──■──»
      ├───┤┌────────────────┐┌────────┐┌─┴─┐┌───────┐┌──────────────┐└─┬─┘     »
q0_3: ┤ X ├┤ Y_2^-1(1.4009) ├┤ Rz(-π) ├┤ X ├┤ Rz(π) ├┤ Y_2(-1.4009) ├──■───────»
      └───┘└────────────────┘└────────┘└───┘└───────┘└──────────────┘          »
q0_4: ─────────────────────────────────────────────────────────────────────────»
                                                                               »
c0: 4/═════════════════════════════════════════════════════════════════════════»
                                                                               »
«                                                                       ┌───┐»
«q0_0: ─────────────────────────────────────────────────────────────────┤ X ├»
«                                                                  ┌───┐└─┬─┘»
«q0_1: ───────────────────────────────■────────────────────────────┤ X ├──■──»
«      ┌─────────────────┐┌────────┐┌─┴─┐┌───────┐┌───────────────┐└─┬─┘┌───┐»
«q0_2: ┤ Y_3^-1(0.61548) ├┤ Rz(-π) ├┤ X ├┤ Rz(π) ├┤ Y_3(-0.61548) ├──■──┤ X ├»
«      └─────────────────┘└────────┘└───┘└───────┘└───────────────┘     └─┬─┘»
«q0_3: ───────────────────────────────────────────────────────────────────■──»
«                                                                            »
«q0_4: ──────────────────────────────────────────────────────────────────────»
«                                                                            »
«c0: 4/══════════════════════════════════════════════════════════════════════»
«                                                                            »
«                                                                ┌───┐     »
«q0_0: ───────────────────────────────■──────────────────────────┤ X ├─────»
«       ┌───────────────┐ ┌────────┐┌─┴─┐┌───────┐ ┌───────────┐ └─┬─┘┌───┐»
«q0_1: ─┤ Y_4^-1(-3π/2) ├─┤ Rz(-π) ├┤ X ├┤ Rz(π) ├─┤ Y_4(3π/2) ├───■──┤ X ├»
«       └───────────────┘ └────────┘└───┘└───────┘ └───────────┘ ┌───┐└─┬─┘»
«q0_2: ───────────────────────────────■──────────────────────────┤ X ├──■──»
«      ┌─────────────────┐┌────────┐┌─┴─┐┌───────┐┌─────────────┐└─┬─┘     »
«q0_3: ┤ Y_5^-1(-6.8068) ├┤ Rz(-π) ├┤ X ├┤ Rz(π) ├┤ Y_5(6.8068) ├──■───────»
«      └─────────────────┘└────────┘└───┘└───────┘└─────────────┘          »
«q0_4: ────────────────────────────────────────────────────────────────────»
«                                                                          »
«c0: 4/════════════════════════════════════════════════════════════════════»
«                                                                          »
«                                                                       
«q0_0: ─────────────────────────────────────────────────────────────────
«                                                                  ┌───┐
«q0_1: ───────────────────────────────■────────────────────────────┤ X ├
«      ┌─────────────────┐┌────────┐┌─┴─┐┌───────┐┌───────────────┐└─┬─┘
«q0_2: ┤ Y_3^-1(0.61548) ├┤ Rz(-π) ├┤ X ├┤ Rz(π) ├┤ Y_3(-0.61548) ├──■──
«      └─────────────────┘└────────┘└───┘└───────┘└───────────────┘     
«q0_3: ─────────────────────────────────────────────────────────────────
«                                                                       
«q0_4: ─────────────────────────────────────────────────────────────────
«                                                                       
«c0: 4/═════════════════════════════════════════════════════════════════
«

In [13]:
circ.measure(qubits,[0,1,2,3])

In [14]:
job = execute(circ, backend = backend, shots = shots, optimization_level=0)
direct_counts = job.result().get_counts()

mitigated_counts, meas_filter  = mitigation(backend = backend, backend_name = backend_name, n=n, qubits = qubits, shots = shots, 
                                       time = "2021.02.16", filename = "test_mitigating",
                                       direct_counts = direct_counts)

TypeError: mitigation() got an unexpected keyword argument 'backend_name'

In [15]:
saver(filename = "Test", time=time,backend_name = backend_name, shots = shots, qubits = qubits,
     direct_counts=direct_counts, mitigated_counts=mitigated_counts, meas_filter=meas_filter)

NameError: name 'mitigated_counts' is not defined

## Casablanca

In [ ]:
backend = provider.get_backend('ibmq_casablanca')
backend_name = 'Casablanca'

time = "2021.04.26"
n=7
shots = 8192

filename = "Casablanca0"
qubits = np.array([0,1,3,5])

In [ ]:
circ,qreg,creg = circ_maker(n = n, qubits=qubits, f =False)
initial_layout = {
    qreg[0]: 0,
    qreg[1]: 1,
    qreg[2]: 2,
    qreg[3]: 3,
    qreg[4]: 4,
    qreg[5]: 5,
    qreg[6]: 6,   
}

tr_circ = transpile(circ ,backend = backend, optimization_level = 3, initial_layout = initial_layout)

tr_circ.draw()

In [ ]:
circ.draw()

In [ ]:
tr_circ.measure(qubits,[0,1,2,3])

In [ ]:
job = execute(tr_circ, backend = backend, shots = shots, optimization_level=0)
direct_counts = job.result().get_counts()

mitigated_counts, meas_filter  = mitigation(backend = backend, n=n, qubits = qubits, 
                                            shots = shots, direct_counts = direct_counts)

In [ ]:
class_fid(direct_counts, comp_to_Cab=True)

In [ ]:
class_fid(mitigated_counts, comp_to_Cab=True)

In [ ]:
saver(filename = filename, time=time,backend_name = backend_name, shots = shots, qubits = qubits,
     direct_counts=direct_counts, mitigated_counts=mitigated_counts, meas_filter=meas_filter)

## Athens

In [ ]:
backend = provider.get_backend('ibmq_athens')
backend_name = 'Athens'

time = "2021.04.23"
n=5
shots = 8192

filename = "Athens3"
qubits = np.array([3,2,1,0])

In [ ]:
circ,qreg,creg = circ_maker(n = n, qubits=qubits, f =False)

initial_layout = {
    qreg[0]: 0,
    qreg[1]: 1,
    qreg[2]: 2,
    qreg[3]: 3,
    qreg[4]: 4,  
}



tr_circ = transpile(circ ,backend = backend, optimization_level = 3, initial_layout=initial_layout)

tr_circ.draw()

In [ ]:
circ.draw()

In [ ]:
tr_circ.measure(qubits,[0,1,2,3])

In [ ]:
job = execute(tr_circ, backend = backend, shots = shots, optimization_level=0)
direct_counts = job.result().get_counts()

mitigated_counts, meas_filter  = mitigation(backend = backend, n=n, qubits = qubits, 
                                            shots = shots, direct_counts = direct_counts)

In [ ]:
class_fid(direct_counts, comp_to_Cab=True)

In [ ]:
class_fid(mitigated_counts, comp_to_Cab=True)

In [ ]:
saver(filename = filename, time=time,backend_name = backend_name, shots = shots, qubits = qubits,
     direct_counts=direct_counts, mitigated_counts=mitigated_counts, meas_filter=meas_filter)

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-c3ow" rowspan="2">qubits</th>
    <th class="tg-c3ow" colspan="2">old<br></th>
    <th class="tg-c3ow" colspan="2">new</th>
  </tr>
  <tr>
    <td class="tg-c3ow">direct</td>
    <td class="tg-c3ow">mitigated</td>
    <td class="tg-c3ow">direct</td>
    <td class="tg-c3ow">mitigated</td>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-c3ow">0-1-2-3</td>
    <td class="tg-c3ow">0.8657</td>
    <td class="tg-c3ow">0.8984</td>
    <td class="tg-c3ow">0.8852</td>
    <td class="tg-c3ow">0.9149</td>
  </tr>
  <tr>
    <td class="tg-c3ow">1-2-3-4</td>
    <td class="tg-c3ow">0.8666</td>
    <td class="tg-c3ow">0.9123</td>
    <td class="tg-c3ow">0.8972</td>
    <td class="tg-c3ow">0.9278</td>
  </tr>
  <tr>
    <td class="tg-c3ow">3-2-1-0</td>
    <td class="tg-c3ow">0.853</td>
    <td class="tg-c3ow">0.8877</td>
    <td class="tg-c3ow">0.8767</td>
    <td class="tg-c3ow">0.907</td>
  </tr>
  <tr>
    <td class="tg-c3ow">4-3-2-1</td>
    <td class="tg-c3ow">0.8417</td>
    <td class="tg-c3ow">0.8802</td>
    <td class="tg-c3ow">0.8958</td>
    <td class="tg-c3ow">0.926</td>
  </tr>
</tbody>
</table>

## Santiago

In [ ]:
backend = provider.get_backend('ibmq_santiago')
backend_name = 'Santiago'

time = "2021.04.25"
n=5
shots = 8192

filename = "Santiago4"
qubits = np.array([4,3,2,1])

In [ ]:
circ.draw()

In [ ]:
circ,qreg,creg = circ_maker(n = n, qubits=qubits, f =False)
tr_circ = transpile(circ ,backend = backend, optimization_level = 3)

tr_circ.draw()

In [ ]:
tr_circ.measure(qubits,[0,1,2,3])

In [ ]:
job = execute(tr_circ, backend = backend, shots = shots, optimization_level=0)
direct_counts = job.result().get_counts()

mitigated_counts, meas_filter  = mitigation(backend = backend, n=n, qubits = qubits, 
                                            shots = shots, direct_counts = direct_counts)

In [ ]:
class_fid(direct_counts, comp_to_Cab=True)

In [ ]:
class_fid(mitigated_counts, comp_to_Cab=True)

In [ ]:
saver(filename = filename, time=time,backend_name = backend_name, shots = shots, qubits = qubits,
     direct_counts=direct_counts, mitigated_counts=mitigated_counts, meas_filter=meas_filter)

## Rome

In [ ]:
backend = provider.get_backend('ibmq_rome')
backend_name = 'Rome'

time = "2021.04.26"
n=5
shots = 8192

filename = "Rome4"
qubits = np.array([4,3,2,1])

In [ ]:
circ,qreg,creg = circ_maker(n = n, qubits=qubits, f =False)
tr_circ = transpile(circ ,backend = backend, optimization_level = 3)

tr_circ.draw()

In [ ]:
circ.draw()

In [ ]:
tr_circ.measure(qubits,[0,1,2,3])

In [ ]:
job = execute(tr_circ, backend = backend, shots = shots, optimization_level=0)
direct_counts = job.result().get_counts()

mitigated_counts, meas_filter  = mitigation(backend = backend, n=n, qubits = qubits, 
                                            shots = shots, direct_counts = direct_counts)

In [ ]:
class_fid(direct_counts, comp_to_Cab=True)

In [ ]:
class_fid(mitigated_counts, comp_to_Cab=True)

In [ ]:
saver(filename = filename, time=time,backend_name = backend_name, shots = shots, qubits = qubits,
     direct_counts=direct_counts, mitigated_counts=mitigated_counts, meas_filter=meas_filter)

## Bogota

In [ ]:
backend = provider.get_backend('ibmq_bogota')
backend_name = 'Bogota'

time = "2021.04.26"
n=5
shots = 8192

filename = "Bogota4"
qubits = np.array([4,3,2,1])

In [ ]:
circ,qreg,creg = circ_maker(n = n, qubits=qubits, f =False)
tr_circ = transpile(circ ,backend = backend, optimization_level = 3)

tr_circ.draw()

In [ ]:
circ.draw()

In [ ]:
tr_circ.measure(qubits,[0,1,2,3])

In [ ]:
job = execute(tr_circ, backend = backend, shots = shots, optimization_level=0)
direct_counts = job.result().get_counts()

mitigated_counts, meas_filter  = mitigation(backend = backend, n=n, qubits = qubits, 
                                            shots = shots, direct_counts = direct_counts)

In [ ]:
class_fid(direct_counts, comp_to_Cab=True)

In [ ]:
class_fid(mitigated_counts, comp_to_Cab=True)

In [ ]:
saver(filename = filename, time=time,backend_name = backend_name, shots = shots, qubits = qubits,
     direct_counts=direct_counts, mitigated_counts=mitigated_counts, meas_filter=meas_filter)

## Manila

In [17]:
backend = provider.get_backend('ibmq_manila')
backend_name = 'Manila'

time = "2021.05.26"
n=5
shots = 8192

filename = "Manila4"
qubits = np.array([4,3,2,1])

In [21]:
circ,qreg,creg = circ_maker(n = n, qubits=qubits, f =False)
tr_circ = transpile(circ ,backend = backend, optimization_level = 3)

tr_circ.draw()

global phase: π
                                                                              »
q160_0 -> 0 ──────────────────────────────────────────────────────────────────»
            ┌────┐┌────────────┐┌────┐┌───────┐┌───┐┌────┐┌────────────┐┌────┐»
q160_1 -> 1 ┤ √X ├┤ RZ(4.8823) ├┤ √X ├┤ RZ(π) ├┤ X ├┤ √X ├┤ RZ(4.5425) ├┤ √X ├»
            ├───┬┘└────────────┘└────┘└───────┘└─┬─┘└────┘└────────────┘└────┘»
q160_2 -> 2 ┤ X ├────────────────────────────────■────────────────────────────»
            ├───┴┐ ┌──────────┐ ┌────┐┌───────┐┌───┐┌────┐ ┌──────────┐ ┌────┐»
q160_3 -> 3 ┤ √X ├─┤ RZ(5π/4) ├─┤ √X ├┤ RZ(π) ├┤ X ├┤ √X ├─┤ RZ(7π/4) ├─┤ √X ├»
            ├───┬┘ └──────────┘ └────┘└───────┘└─┬─┘└────┘ └──────────┘ └────┘»
q160_4 -> 4 ┤ X ├────────────────────────────────■────────────────────────────»
            └───┘                                                             »
      c5: 4/══════════════════════════════════════════════════════════════════»
                                                                              »
«                                                                 »
«q160_0 -> 0 ─────────────────────────────────────────────────────»
«                                                                 »
«q160_1 -> 1 ──■──────────────────────────────────────────────────»
«            ┌─┴─┐           ┌────┐┌────────────┐┌────┐┌───┐┌────┐»
«q160_2 -> 2 ┤ X ├─────■─────┤ √X ├┤ RZ(3.7571) ├┤ √X ├┤ X ├┤ √X ├»
«            └───┘   ┌─┴─┐   └────┘└────────────┘└────┘└─┬─┘└────┘»
«q160_3 -> 3 ──■─────┤ X ├───────────────────────────────■────────»
«            ┌─┴─┐┌──┴───┴──┐┌────┐ ┌──────────┐                  »
«q160_4 -> 4 ┤ X ├┤ RZ(π/2) ├┤ √X ├─┤ RZ(3π/2) ├──────────────────»
«            └───┘└─────────┘└────┘ └──────────┘                  »
«      c5: 4/═════════════════════════════════════════════════════»
«                                                                 »
«                                                                 »
«q160_0 -> 0 ─────────────────────────────────────────────────────»
«                                                 ┌────────┐┌────┐»
«q160_1 -> 1 ──────────────────────────────■──────┤ RZ(-π) ├┤ √X ├»
«            ┌────────────┐┌────┐        ┌─┴─┐    └────────┘└────┘»
«q160_2 -> 2 ┤ RZ(3.7571) ├┤ √X ├──■─────┤ X ├────────────────────»
«            └────────────┘└────┘┌─┴─┐┌──┴───┴───┐  ┌────┐  ┌────┐»
«q160_3 -> 3 ────────────────────┤ X ├┤ RZ(-π/2) ├──┤ √X ├──┤ √X ├»
«                                └───┘└──────────┘  └────┘  └────┘»
«q160_4 -> 4 ─────────────────────────────────────────────────────»
«                                                                 »
«      c5: 4/═════════════════════════════════════════════════════»
«                                                                 »
«                                                                   »
«q160_0 -> 0 ───────────────────────────────────────────────────────»
«            ┌────────────┐┌────┐ ┌───────┐  ┌───┐  ┌────────┐┌────┐»
«q160_1 -> 1 ┤ RZ(3.6652) ├┤ √X ├─┤ RZ(π) ├──┤ X ├──┤ RZ(-π) ├┤ √X ├»
«            └────────────┘└────┘ └───────┘  └─┬─┘  └────────┘└────┘»
«q160_2 -> 2 ──────────────────────────────────■────────────────────»
«             ┌──────────┐          ┌────┐   ┌────┐ ┌───────┐       »
«q160_3 -> 3 ─┤ RZ(3π/2) ├───■──────┤ √X ├───┤ √X ├─┤ RZ(π) ├───────»
«             └──────────┘ ┌─┴─┐ ┌──┴────┴──┐├────┤┌┴───────┴┐      »
«q160_4 -> 4 ──────────────┤ X ├─┤ RZ(3π/2) ├┤ √X ├┤ RZ(π/2) ├──────»
«                          └───┘ └──────────┘└────┘└─────────┘      »
«      c5: 4/═══════════════════════════════════════════════════════»
«                                                                   »
«                                                                             »
«q160_0 -> 0 ─────────────────────────────────────────────────────────────────»
«            ┌────────────┐┌────┐┌───────┐                                    »
«q160_1 -> 1 ┤ RZ(3.6652) 

In [22]:
tr_circ.measure(qubits,[0,1,2,3])

In [23]:
job = execute(tr_circ, backend = backend, shots = shots, optimization_level=0)
direct_counts = job.result().get_counts()

mitigated_counts, meas_filter  = mitigation(backend = backend, n=n, qubits = qubits, 
                                            shots = shots, direct_counts = direct_counts)

In [24]:
class_fid(direct_counts, comp_to_Cab=True)

0.8788736009259748

In [25]:
class_fid(mitigated_counts, comp_to_Cab=True)

0.911096674077179

In [26]:
saver(filename = filename, time=time,backend_name = backend_name, shots = shots, qubits = qubits,
     direct_counts=direct_counts, mitigated_counts=mitigated_counts, meas_filter=meas_filter)